In [1]:
import numpy as np
import pandas as pd

# Load Huggingface transformers
from transformers import TFBertModel,  BertConfig, BertTokenizerFast

# Then what you need from tensorflow.keras
from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical

# And pandas for data import + sklearn because you allways need sklearn
import pandas as pd
from sklearn.model_selection import train_test_split

pip install torch===1.6.0 torchvision===0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

pip install tqdm -U

In [2]:
import torch
from tqdm.notebook import tqdm

from transformers import BertTokenizer
from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification


In [3]:
data1 = pd.read_excel (r'./Matric_Couverture3_evolution.xlsx', index_col=None, header=None)
data = pd.DataFrame(data1)
data = data.iloc[0:2]
data=data.transpose()
data = data[[1, 0]]
data.columns = ["texte", "Evolution"]
data=data.drop(index=0)
data

,texte,Evolution
1,le système affiche les boutons des réseaux soc...,0
2,"sur le logo depuis une page intérieure, le sys...",0
3,"Sur le logo depuis la page daccueil, le systè...",2
4,Si lutilisateur clique sur le bouton daccess...,0
5,Un clic sur un bouton de réseaux sociaux perme...,0
6,Le système affiche :,3
7,"Au clic sur la langue sélectionnée, le système...",0
8,"Si la langue sélectionnée est français, le sys...",0
9,"Si la langue sélectionnée est anglais, le syst...",0
10,"Si la langue sélectionnée est espagnol, le sys...",0


In [4]:
data['Evolution'].value_counts()

1    535
0    436
2    156
3     30
Name: Evolution, dtype: int64

In [5]:
possible_labels = data.Evolution.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = possible_label
label_dict

{0: 0, 2: 2, 3: 3, 1: 1}

In [6]:
data['label'] = data.Evolution.replace(label_dict)
data.head()

,texte,Evolution,label
1,le système affiche les boutons des réseaux soc...,0,0
2,"sur le logo depuis une page intérieure, le sys...",0,0
3,"Sur le logo depuis la page daccueil, le systè...",2,2
4,Si lutilisateur clique sur le bouton daccess...,0,0
5,Un clic sur un bouton de réseaux sociaux perme...,0,0


In [7]:
data.index.values

array([   1,    2,    3, ..., 1155, 1156, 1157], dtype=int64)

X_train, X_test, y_train, y_test = train_test_split(data.texte.values, data.Evolution.values.astype('category'), test_size=0.2, random_state=0)

In [8]:

from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(data.index.values, 
                                                  data.label.values, 
                                                  test_size=0.2, 
                                                  random_state=42, 
                                                  stratify=data.label.values)

data['data_type'] = ['not_set']*data.shape[0]

data.loc[X_train, 'data_type'] = 'train'
data.loc[X_val, 'data_type'] = 'val'

data.groupby(['Evolution', 'label', 'data_type']).count()

texte
Evolution label data_type       
0         0     train        348
                val           88
1         1     train        428
                val          107
2         2     train        125
                val           31
3         3     train         24
                val            6

In [9]:
data.head()

,texte,Evolution,label,data_type
1,le système affiche les boutons des réseaux soc...,0,0,train
2,"sur le logo depuis une page intérieure, le sys...",0,0,val
3,"Sur le logo depuis la page daccueil, le systè...",2,2,train
4,Si lutilisateur clique sur le bouton daccess...,0,0,train
5,Un clic sur un bouton de réseaux sociaux perme...,0,0,train


In [10]:
#data.loc[data['data_type'] == "train" and data['Evolution'] == '0':]
df0 = data[(data['data_type'] == "train") & (data['label'] ==0 )].head(100)
df0

,texte,Evolution,label,data_type
1,le système affiche les boutons des réseaux soc...,0,0,train
4,Si lutilisateur clique sur le bouton daccess...,0,0,train
5,Un clic sur un bouton de réseaux sociaux perme...,0,0,train
7,"Au clic sur la langue sélectionnée, le système...",0,0,train
8,"Si la langue sélectionnée est français, le sys...",0,0,train
9,"Si la langue sélectionnée est anglais, le syst...",0,0,train
10,"Si la langue sélectionnée est espagnol, le sys...",0,0,train
11,"Au clic sur la deuxième ou troisième langue, l...",0,0,train
12,"Si la page est traduite, celle-ci est affichée.",0,0,train
16,Le système affiche pour chaque langue ci-desso...,0,0,train


In [11]:
df1 = data[(data['data_type'] == "train") & (data['label'] ==1 )].head(100)
df1

,texte,Evolution,label,data_type
13,\tle système redirige vers la page parente tr...,1,1,train
14,\tSi la page parente de niveau N-1 nest pas ...,1,1,train
24,Date / Horaires,1,1,train
28,Au clic sur lune des entrées de niveau 1,1,1,train
29,"Au clic sur licône Etoile, lutilisateur est ...",1,1,train
32,Lutilisateur peut créer un lien de niveau 1,1,1,train
33,Lutilisateur peut créer un lien de niveau 2,1,1,train
34,Une page nouvellement créée en back-office est...,1,1,train
36,Si lutilisateur a sélectionné anglais ou espa...,1,1,train
37,Aucun contenu en français nest affiché,1,1,train


In [12]:
df2 = data[(data['data_type'] == "train") & (data['label'] ==2 )].head(100)
df2

,texte,Evolution,label,data_type
3,"Sur le logo depuis la page daccueil, le systè...",2,2,train
15,\tSi la page parente de niveau N-2 nest pas ...,2,2,train
43,Informations pratiques,2,2,train
48,Après avoir effectué une recherche dans lagen...,2,2,train
49,"Après avoir effectué une recherche, la page de...",2,2,train
50,"Après avoir affiché les infos pratiques, le po...",2,2,train
85,"Dans le bloc de contenu mis en avant, le syst...",2,2,train
86,"Dans la barre de navigation, le système affiche",2,2,train
95,Cas dutilisation permettant à un internaute ...,2,2,train
102,\tUn lien En savoir plus,2,2,train


In [13]:
df3 = data[(data['data_type'] == "train") & (data['label'] ==3 )]
df3

,texte,Evolution,label,data_type
6,Le système affiche :,3,3,train
45,Informations pratiques (voir cas dutilisation...,3,3,train
57,Un bouton En savoir plus,3,3,train
137,\tLe bloc En savoir plus (voir cas dutilisat...,3,3,train
138,\tLe bloc Partenaires (voir cas dutilisation...,3,3,train
144,Seul le premier bloc éditorial est affiché,3,3,train
167,"Au clic sur un visuel dune notice, le système...",3,3,train
169,Un bouton en savoir plus,3,3,train
188,Au clic un élément,3,3,train
250,Voir cas dutilisation Contribuer un partenaire,3,3,train


In [14]:
dfv = data[(data['data_type'] == "val")]
dfv

,texte,Evolution,label,data_type
2,"sur le logo depuis une page intérieure, le sys...",0,0,val
18,Nom du champ\t\tType de champ\tobligatoire\tCo...,2,2,val
20,Le fil dariane est composé dun lien vers la ...,0,0,val
25,Infos pratiques,1,1,val
31,Les rubriques de niveau 3 saffichent et sont ...,0,0,val
35,2 boutons de pages peuvent être contribués pou...,0,0,val
38,Si lutilisateur a sélectionné une langue autr...,1,1,val
46,"Au clic sur licône « croix » de fermeture, la...",2,2,val
55,Le contributeur peut ajouter toute page dans l...,1,1,val
59,Le bouton En savoir plus,3,3,val


In [15]:
d = pd.DataFrame(columns=['texte','Evolution','label','data_type'])
d

,texte,Evolution,label,data_type


In [16]:
x=data.loc[2:2]
d=pd.concat([d,df0])
d=pd.concat([d,df1])
d=pd.concat([d,df2])
d=pd.concat([d,df3])
d=pd.concat([d,df3])
d=pd.concat([d,df3])
d=pd.concat([d,dfv])
d

,texte,Evolution,label,data_type
1,le système affiche les boutons des réseaux soc...,0,0,train
4,Si lutilisateur clique sur le bouton daccess...,0,0,train
5,Un clic sur un bouton de réseaux sociaux perme...,0,0,train
7,"Au clic sur la langue sélectionnée, le système...",0,0,train
8,"Si la langue sélectionnée est français, le sys...",0,0,train
9,"Si la langue sélectionnée est anglais, le syst...",0,0,train
10,"Si la langue sélectionnée est espagnol, le sys...",0,0,train
11,"Au clic sur la deuxième ou troisième langue, l...",0,0,train
12,"Si la page est traduite, celle-ci est affichée.",0,0,train
16,Le système affiche pour chaque langue ci-desso...,0,0,train


In [17]:
data = d
data

,texte,Evolution,label,data_type
1,le système affiche les boutons des réseaux soc...,0,0,train
4,Si lutilisateur clique sur le bouton daccess...,0,0,train
5,Un clic sur un bouton de réseaux sociaux perme...,0,0,train
7,"Au clic sur la langue sélectionnée, le système...",0,0,train
8,"Si la langue sélectionnée est français, le sys...",0,0,train
9,"Si la langue sélectionnée est anglais, le syst...",0,0,train
10,"Si la langue sélectionnée est espagnol, le sys...",0,0,train
11,"Au clic sur la deuxième ou troisième langue, l...",0,0,train
12,"Si la page est traduite, celle-ci est affichée.",0,0,train
16,Le système affiche pour chaque langue ci-desso...,0,0,train


In [18]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
                                          
encoded_data_train = tokenizer.batch_encode_plus(
    data[data.data_type=='train'].texte.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=100, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    data[data.data_type=='val'].texte.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
C:\Users\bigdata\Anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:1773: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
Truncation was not explicitely activated but `max_length` is provided a specific value, plea

In [19]:
data["label"] = data["label"].astype(np.int64)
#data.astype({'label': 'int64'}).dtypes

data.label.head()


1    0
4    0
5    0
7    0
8    0
Name: label, dtype: int64

In [20]:
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(data[data.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(data[data.data_type=='val'].label.values)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [21]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [22]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 3

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [23]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)
                  
epochs = 10

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [24]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [25]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals
    
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        use_cuda =  torch.cuda.is_available()
        device = torch.device("cuda" if use_cuda else "cpu")
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'data_volume/finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')


Epoch 1
Training loss: 1.3710962466655239
Validation loss: 1.276794699522165
F1 Score (Weighted): 0.2736697483719675



Epoch 2
Training loss: 1.2404054624419059
Validation loss: 1.2303666075070698
F1 Score (Weighted): 0.2696012256182079



Epoch 3
Training loss: 1.0268670032101292
Validation loss: 1.2570020800981767
F1 Score (Weighted): 0.3891729542599811



Epoch 4
Training loss: 0.8794768140200646
Validation loss: 1.3536369250370905
F1 Score (Weighted): 0.3490729188219856



Epoch 5
Training loss: 0.7640807491637045
Validation loss: 1.5383665630450616
F1 Score (Weighted): 0.2899861100880747



Epoch 6
Training loss: 0.6083597528838343
Validation loss: 1.6757206853765707
F1 Score (Weighted): 0.32404851971519966



Epoch 7
Training loss: 0.490058405745414
Validation loss: 1.67763541218562
F1 Score (Weighted): 0.36090178631679415



Epoch 8
Training loss: 0.4154630478231176
Validation loss: 1.9140909961782968
F1 Score (Weighted): 0.35386465788208527



Epoch 9
Training loss: 0.3391816739834124
Validation loss: 2.028237210252346
F1 Score (Weighted): 0.355926877863383



Epoch 10
Training loss: 0.2940098049810096
Validation loss: 2.0929444088385654
F1 Score (Weighted): 0.34012899844025674



In [26]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

model.load_state_dict(torch.load('data_volume/finetuned_BERT_epoch_1.model', map_location=torch.device('cpu')))

_, predictions, true_vals = evaluate(dataloader_validation)
accuracy_per_class(predictions, true_vals)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Class: 0
Accuracy: 27/88

Class: 1
Accuracy: 17/107

Class: 2
Accuracy: 17/31

Class: 3
Accuracy: 1/6

